In [1]:
import os
import re
import json
import requests
from typing import Any

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://pl.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikiprojekt:Czy_wiesz/archiwum'
OUTPUT_DIR = 'data/pl'

MONTH_NUMBERS_TO_NAMES = {
    '01': 'styczeń',
    '02': 'luty',
    '03': 'marzec',
    '04': 'kwiecień',
    '05': 'maj',
    '06': 'czerwiec',
    '07': 'lipiec',
    '08': 'sierpień',
    '09': 'wrzesień',
    '10': 'październik',
    '11': 'listopad',
    '12': 'grudzień',
}


def get_month_links_from_archive(main_page_url):
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Find tables with month links
    t1 = soup.find('table', class_='a1 inner-standard')
    if not t1:
        raise ValueError("Could not find the table with class 'a1 inner-standard'.")
    t2 = soup.find('table', class_='a2 inner-standard')
    if not t2:
        raise ValueError("Could not find the table with class 'a2 inner-standard'.")
    t3 = soup.find('table', class_='a3 inner-standard')
    if not t3:
        raise ValueError("Could not find the table with class 'a3 inner-standard'.")
    t4 = soup.find('table', class_='a4 inner-standard')
    if not t4:
        raise ValueError("Could not find the table with class 'a4 inner-standard'.")

    archive = {}

    # Years 2004-2007
    for tr in t1.find_all('tr'):
        header = tr.find('th').get_text(strip=True)
        date_parts = header.split(':')[0].split('-')
        year, month = date_parts[0], date_parts[1]
        month_name = MONTH_NUMBERS_TO_NAMES[month]

        a = tr.find('a')
        url = a['href']
        full_url = urljoin(BASE_URL, url)
        exists = 'new' not in a.get('class', [])

        if year not in archive:
            archive[year] = []

        archive[year].append({
            'month': month_name,
            'url': full_url,
            'exists': exists
        })

    # Years 2008-2025 (2025 is not complete)
    for ti in [t2, t3, t4]:
        for tr in ti.find_all('tr'):
            year = tr.find('th').get_text(strip=True)

            month_list = tr.find('ul')
            for li in month_list.find_all('li'):
                a = li.find('a')
                month_name = a.get_text(strip=True)
                url = a['href']
                full_url = urljoin(BASE_URL, url)
                exists = 'new' not in a.get('class', [])

                if year not in archive:
                    archive[year] = []

                archive[year].append({
                    'month': month_name,
                    'url': full_url,
                    'exists': exists
                })
     
    return archive        


In [2]:
archive = get_month_links_from_archive(MAIN_URL)

archive

{'2004': [{'month': 'lipiec',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2004-07-13:2005-09-25',
   'exists': True}],
 '2005': [{'month': 'wrzesień',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2005-09-26:2006-06-03',
   'exists': True}],
 '2006': [{'month': 'czerwiec',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2006-06-04:2006-09-21',
   'exists': True},
  {'month': 'wrzesień',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2006-09-22:2007-01-19',
   'exists': True}],
 '2007': [{'month': 'styczeń',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2007-01-20:2007-04-04',
   'exists': True},
  {'month': 'kwiecień',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2007-04-05:2007-05-08',
   'exists': True},
  {'month': 'maj',
   'url': 'https://pl.wikipedia.org/wiki/Wikiprojekt:Czy_wiesz/archiwum/2007-05-09:2007-12-31',
   'exists': 

In [41]:
def _extract_fact_data(element: Tag, base_url: str) -> dict[str, Any]:
    """Extracts text, links, and relevant links from a BeautifulSoup Tag."""
    raw_text = element.get_text(" ", strip=True).replace('Czy wiesz ...', '').lstrip().lstrip('.').lstrip('…').lstrip()
    if not raw_text:
        return {}
    if (
        raw_text.startswith('Strona ekspozycji') or
        raw_text.startswith('Z nowych artykułów w Wikipedii') or
        raw_text.startswith('Z nowych i ostatnio rozbudowanych artykułów')
    ):
        return {}
    
    fact_text = "Czy wiesz " + raw_text
    fact_text = fact_text.replace('  ', ' ')
    fact_text = fact_text.replace('\xa0', ' ')

    links = []
    relevant_links = []
    for a in element.find_all("a", href=True):
        href = a["href"]
        if not href.startswith("/wiki/"):
            continue

        full_url = unquote(urljoin(base_url, href))

        if a.find_parent('b'):
            relevant_links.append(full_url)

        links.append(full_url)

    return {
        "text": fact_text,
        "links": links,
        "relevant_links": relevant_links,
    }


def parse_month_facts(month_url: str, month_name: str, year: str) -> list[dict]:
    resp = requests.get(month_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    # Different parsing for different years
    if int(year) < 2007 or (int(year) == 2007 and month_name != 'maj'):
        main_div = soup.find("div", class_="mw-heading mw-heading2 ext-discussiontools-init-section")
        div = main_div.find_next_sibling("div")

        for p in div.find_all("p"):
            fact_data = _extract_fact_data(p, month_url)

            if fact_data and fact_data['text']:
                results.append({
                    "section": month_name + ' ' + year,
                    **fact_data
                })
    elif int(year) == 2007 and month_name == 'maj':
        for main_div in soup.find_all("div", class_="mw-heading mw-heading2 ext-discussiontools-init-section"):
            section = main_div.find('h2').get_text(strip=True)

            for sib in main_div.find_next_siblings():
                if sib.name == "div" and "mw-heading" in sib.get("class", []):
                    break
                
                if sib.name == 'p':
                    fact_data = _extract_fact_data(sib, month_url)
                    if fact_data and fact_data['text']:
                        results.append({
                            "section": section,
                            **fact_data
                        })
    # For years > 2007
    else:
        for main_div in soup.find_all("div", class_="mw-heading mw-heading3"):
            section = main_div.find('h3').get_text(strip=True)

            for sib in main_div.find_next_siblings():
                if sib.name == "div" and "mw-heading" in sib.get("class", []):
                    break
                
                if sib.name == 'p':
                    fact_data = _extract_fact_data(sib, month_url)
                    
                    if fact_data and fact_data['text']:
                        results.append({
                            "section": section,
                            **fact_data
                        })

    return results


In [44]:
year = '2025'
elem = 7
month = archive[year][elem]['month']

facts = parse_month_facts(archive[year][elem]['url'], month, year)
facts

[{'section': '2025-08-01',
  'text': 'Czy wiesz kto był przywódcą Asasynów? (na zdjęciu cosplay)',
  'links': ['https://pl.wikipedia.org/wiki/Ezio_Auditore_da_Firenze'],
  'relevant_links': ['https://pl.wikipedia.org/wiki/Ezio_Auditore_da_Firenze']},
 {'section': '2025-08-01',
  'text': 'Czy wiesz kim był Widzący z Lublina ?',
  'links': ['https://pl.wikipedia.org/wiki/Jakub_Izaak_Horowic'],
  'relevant_links': ['https://pl.wikipedia.org/wiki/Jakub_Izaak_Horowic']},
 {'section': '2025-08-01',
  'text': 'Czy wiesz co robi w życiu Marian Huja ?',
  'links': ['https://pl.wikipedia.org/wiki/Marian_Huja'],
  'relevant_links': ['https://pl.wikipedia.org/wiki/Marian_Huja']},
 {'section': '2025-08-01',
  'text': 'Czy wiesz który milioner założył biznes, sprzedając śpiwory?',
  'links': ['https://pl.wikipedia.org/wiki/Igor_Klaja'],
  'relevant_links': ['https://pl.wikipedia.org/wiki/Igor_Klaja']},
 {'section': '2025-08-01',
  'text': 'Czy wiesz że najmłodsza deputowana 41. kadencji bułgarskiego

In [53]:
MONTHS = {
    'січень': 'січень', 'січня': 'січень',
    'лютий': 'лютий', 'лютого': 'лютий',
    'березень': 'березень', 'березня': 'березень',
    'квітень': 'квітень', 'квітня': 'квітень',
    'травень': 'травень', 'травня': 'травень',
    'червень': 'червень', 'червня': 'червень',
    'липень': 'липень', 'липня': 'липень',
    'серпень': 'серпень', 'серпня': 'серпень',
    'вересень': 'вересень', 'вересня': 'вересень',
    'жовтень': 'жовтень', 'жовтня': 'жовтень',
    'листопад': 'листопад', 'листопада': 'листопад',
    'грудень': 'грудень', 'грудня': 'грудень',
}

def extract_month_from_title(title: str) -> str | None:
    """Extracts the first mentioned month from a section title."""
    words = re.split(r'[\s—-]+', title)

    for word in words:
        cleaned_word = word.lower().strip()
        
        if cleaned_word in MONTHS:
            return MONTHS[cleaned_word]

    return None

In [60]:
extract_month_from_title('3 22 грудня 2022 — 23 січня 2023')

'грудень'

In [52]:
import dateparser

year = '2025'
month = '7'
section = '11 — 25 лютого 2025'

dt_month = dateparser.parse(section)
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-02'

In [1]:
a = '2015-07-27'
a.split('-')

['2015', '07', '27']

In [ ]:
''